In [ ]:
%matplotlib inline

This notebook is aimed at getting all the effector candidates from the Pst_104E_genome haplotigs as defined as the following.
EffectorP prediction done by Jana Sperschneider
Gene expression cluster analysis done by Jana Sperschneider picking cluster 11, 13, 14, and 15

In [ ]:
import os
from Bio import SeqIO
import pandas as pd
import re
from pybedtools import BedTool
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#define your input folders updated for haplotigs
CLUSTER_FOLDER = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/Pst_104E_genome/gene_expression/Pst104_h_SecretomeClustering'
EFFECTORP_FOLDER = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/Pst_104E_genome/Secretomes/EffectorP'
GFF_FOLDER = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/032017_assembly'
PROTEIN_ANNO_FOLDER = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/enrichment_analysis/pa_26062017'
OUT_FOLDER = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/enrichment_analysis/lists'

In [ ]:
genome = 'Pst_104E_v12_'
h_effector_list = []

In [ ]:
#define what you want to take
clusters = [ 11, 13, 14, 15]
clusters_files = [os.path.join(CLUSTER_FOLDER, x) for x in os.listdir(CLUSTER_FOLDER)\
                 if x.startswith('Cluster') and x.endswith('_DEs.fasta') and\
                  any(str(y) in x for y in clusters) ] #fixed to check if any of the clusters are
                                    #in the file header
effectorp_files = [os.path.join(EFFECTORP_FOLDER, x) for x in os.listdir(EFFECTORP_FOLDER)\
                  if x.endswith('effectors.fasta') and x.startswith(genome)]

In [ ]:
#get all the sequence names into a list from the fasta headers 
for file in clusters_files:
    fh = open(file, 'r')
    for seq in SeqIO.parse(fh, 'fasta'):
        if 'hcontig' in seq.id:
            h_effector_list.append(seq.id)
    fh.close()

for file in effectorp_files:
    fh = open(file, 'r')
    for seq in SeqIO.parse(fh, 'fasta'):
        if 'hcontig' in seq.id:
            h_effector_list.append(seq.id)
    fh.close()

In [ ]:
h_effector_file = os.path.join(OUT_FOLDER, genome + 'h_effector.list')

In [ ]:
#write out the sets of effector candidates
fh = open(h_effector_file, 'w')
for ec in set(h_effector_list):
    print(ec, file=fh)
fh.close()


In [ ]:
#subset the gff files as well and write those out
h_gff_file = [os.path.join(GFF_FOLDER, x) for x in os.listdir(GFF_FOLDER)\
                 if x.startswith(genome+'h_ctg') and x.endswith('anno.gff3') ][0]

In [ ]:
#os.listdir(GFF_FOLDER)

In [ ]:
#get repeat gff files
h_repeat_gff_fn  = [os.path.join(GFF_FOLDER, x) for x in os.listdir(GFF_FOLDER)\
                 if x.startswith(genome+'h_ctg') and x.endswith('REPET.gff') ][0]

In the command line in /home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/TE_analysis the superfamily gff files were sorted as followed and copied over into the lists folder
sort Pst_104E_v12_h_ctg.REPET.superfamily.gff -k1,1n -k4,4n > Pst_104E_v12_h_ctg.REPET.sorted.superfamily.gff

In [ ]:
#get repeat gff files
h_repeat_superfamily_gff_fn  = [os.path.join(OUT_FOLDER, x) for x in os.listdir(OUT_FOLDER)\
                 if x.startswith(genome+'h_ctg') and x.endswith('REPET.sorted.superfamily.gff') ][0]

In [ ]:
#gff header 
gff_header = ['seqid', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase', 'attributes']

In [ ]:
#now subset the gff files for effectors only
h_gff_df = pd.read_csv(h_gff_file, header = None, sep='\t', names= gff_header)
h_gff_df['ID'] = h_gff_df.attributes.str.extract(r'ID=([^;]*);', expand=False)
h_gff_df.sort_values(by=['seqid', 'start'], inplace = True)

In [ ]:
#now sort REPET gff and write out again
h_repeat_gff_df = pd.read_csv(h_repeat_gff_fn, header=None, sep='\t', names=gff_header, comment='#')
h_repeat_gff_fn = os.path.join(OUT_FOLDER,h_repeat_gff_fn.split('/')[-1] )
h_repeat_gff_df.sort_values(by=['seqid', 'start']).to_csv(h_repeat_gff_fn, header=None, index=None, sep='\t')

In [ ]:
#now write out dataframes for h_gff

#bed 6 file
h_effector_bed_fn = h_effector_file.replace('.list', '.gene.bed')
h_gff_df[(h_gff_df.type == 'gene') & (h_gff_df.ID.str.replace('TU', 'model').isin(h_effector_list))].\
    loc[:,['seqid', 'start', 'end', 'ID', 'score', 'strand']].to_csv(h_effector_bed_fn, header=None, index=None, sep='\t')
    
h_effector_gff_fn = h_effector_file.replace('.list', '.gene.gff3')    
h_gff_df[(h_gff_df.type == 'gene') & (h_gff_df.ID.str.replace('TU', 'model').isin(h_effector_list))].\
    loc[:,gff_header].to_csv(h_effector_gff_fn, header=None, index=None, sep='\t')


#bed 6 file no effectors
h_noeffector_bed_fn = h_effector_file.replace('.list', 'h_noeffector.gene.bed')
h_gff_df[(h_gff_df.type == 'gene') & (~h_gff_df.ID.str.replace('TU', 'model').isin(h_effector_list))].\
    loc[:,['seqid', 'start', 'end', 'ID', 'score', 'strand']].to_csv(h_noeffector_bed_fn, header=None, index=None, sep='\t')

#no effector gff and list
h_noeffector_gff_fn = h_effector_file.replace('h_effector.list', 'h_noeffector.gene.gff3')    
h_gff_df[(h_gff_df.type == 'gene') & (~h_gff_df.ID.str.replace('TU', 'model').isin(h_effector_list))].\
    loc[:,gff_header].to_csv(h_noeffector_gff_fn, header=None, index=None, sep='\t')

file_name = h_effector_file.replace('h_effector.list', 'h_noeffector.list')
h_gff_df[(h_gff_df.type == 'gene') & (~h_gff_df.ID.str.replace('TU', 'model').isin(h_effector_list))]['ID'].str.replace('TU', 'model')\
    .to_csv(file_name, header=None, index=None, sep='\t')

In [ ]:
#now get BUSCO list and write out stuff
h_busco_file = [os.path.join(PROTEIN_ANNO_FOLDER, x) for x in os.listdir(PROTEIN_ANNO_FOLDER) if x.startswith(genome+'h_ctg') and 'busco' in x][0]
h_busco_list = pd.read_csv(h_busco_file, header=None, sep='\t')[0].tolist()

In [ ]:
#write out BUSCO for hcontigs
file_name = h_effector_file.replace('effector.list', 'busco.list')
h_gff_df[(h_gff_df.type == 'gene') & (h_gff_df.ID.str.replace('TU', 'model').isin(h_busco_list))]['ID'].str.replace('TU', 'model')\
    .to_csv(file_name, header=None, index=None, sep='\t')
h_busco_gff_fn = h_effector_file.replace('effector.list', 'busco.gene.gff3')
h_gff_df[(h_gff_df.type == 'gene') & (h_gff_df.ID.str.replace('TU', 'model').isin(h_busco_list))].\
    loc[:,gff_header].to_csv(h_busco_gff_fn, header=None, index=None, sep='\t')

h_busco_bed_fn = h_effector_file.replace('effector.list', 'busco.gene.bed')
h_gff_df[(h_gff_df.type == 'gene') & (h_gff_df.ID.str.replace('TU', 'model').isin(h_busco_list))].\
    loc[:,['seqid', 'start', 'end', 'ID', 'score', 'strand']].to_csv(h_busco_bed_fn, header=None, index=None, sep='\t')
    
h_nobusco_bed_fn = h_effector_file.replace('effector.list', 'no_busco.gene.bed')    
h_gff_df[(h_gff_df.type == 'gene') & (~h_gff_df.ID.str.replace('TU', 'model').isin(h_busco_list))].\
    loc[:,['seqid', 'start', 'end', 'ID', 'score', 'strand']].to_csv(h_nobusco_bed_fn, header=None, index=None, sep='\t')
    

In [ ]:
#write out no busco no effector bed files
h_noeffector_nobusco_bed_fn = h_effector_file.replace('effector.list', 'no_busco_no_effector.gene.bed')    
h_gff_df[(h_gff_df.type == 'gene') & (~h_gff_df.ID.str.replace('TU', 'model').isin(h_busco_list))\
         &(~h_gff_df.ID.str.replace('TU', 'model').isin(h_effector_list))].\
    loc[:,['seqid', 'start', 'end', 'ID', 'score', 'strand']].to_csv(h_noeffector_nobusco_bed_fn, header=None, index=None, sep='\t')


In [ ]:
#write all genes
h_gene_gff_fn = h_effector_file.replace('effector.list', 'all.gene.gff3')
h_gff_df[(h_gff_df.type == 'gene') ].\
    loc[:,gff_header].to_csv(h_gene_gff_fn, header=None, index=None, sep='\t')
#write bed 6
h_gene_bed_fn = h_effector_file.replace('effector.list', 'all.gene.bed')
h_gff_df[(h_gff_df.type == 'gene') ].\
    loc[:,['seqid', 'start', 'end', 'ID', 'score', 'strand']].to_csv(h_gene_bed_fn, header=None, index=None, sep='\t')

Fixed it up to here waiting to get the cluster genes for effector annotation


In [ ]:
!head {p_effector_bed_fn}

In [ ]:
#get the distances
p_effector_bed = BedTool(p_effector_gff_fn)
p_noeffector_bed = BedTool(p_noeffector_gff_fn)
p_busco_bed = BedTool(p_busco_gff_fn)
p_repeats_bed = BedTool(p_repeat_superfamily_gff_fn)

p_closest_rep_to_eff = p_effector_bed.closest(p_repeats_bed, d=True)

p_closest_rep_to_eff_df = p_closest_rep_to_eff.to_dataframe()

tmp_REPET = pd.read_csv(p_repeat_superfamily_gff_fn, header=None, sep='\t', names=gff_header)
tmp_REPET['distance'] = tmp_REPET.end - tmp_REPET.start
tmp_fn = p_repeat_superfamily_gff_fn.replace('superfamily', 'tmpsuperfamily')
tmp_REPET[tmp_REPET.distance > 400].loc[:,gff_header].to_csv(tmp_fn, header=None, sep='\t', index=None)

p_repeats_bed = BedTool(tmp_fn)

In [ ]:
#bed closest header
bed_repeat_closest_header = [x +'_gene' for x in gff_header] + [x +'_repeat' for x in gff_header] + ['distance']

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
print(p_effector_bed.closest(p_repeats_bed,d=True,t='last').to_dataframe().iloc[:,18:20].describe())
p_effector_bed.closest(p_repeats_bed, d=True, t='last').to_dataframe().boxplot(column=18)
plt.ylim(0, 10000)

In [ ]:
print(p_busco_bed.closest(p_repeats_bed, d=True,t='last').to_dataframe().iloc[:,18:20].describe())
p_busco_bed.closest(p_repeats_bed, d=True,t='last').to_dataframe().boxplot(column=18)
plt.ylim(0, 10000)

In [ ]:
print(p_noeffector_bed.closest(p_repeats_bed, d=True,t='last').to_dataframe().iloc[:,18:20].describe())
p_noeffector_bed.closest(p_repeats_bed, d=True,t='last').to_dataframe().boxplot(column=18)
plt.ylim(0, 10000)

In [ ]:
p_noeffector_bed.closest(p_repeats_bed, d=True,t='last', io=True).to_dataframe().groupby(17).count()[0].sort_values().tail(10)/len(p_noeffector_bed)*100

In [ ]:
p_effector_bed.closest(p_repeats_bed, d=True,t='last', io=True).to_dataframe().groupby(17).count()[0].sort_values().tail(10)/len(p_effector_bed)*100

In [ ]:
p_busco_bed.closest(p_repeats_bed, d=True,t='last', io=True).to_dataframe().groupby(17).count()[0].sort_values().tail(10)/len(p_busco_bed)*100

In [ ]:
p_busco_bed.closest(p_repeats_bed, d=True,t='last').to_dataframe().groupby(17).mean()[18].sort_values().tail(10)

In [ ]:
p_noeffector_bed.closest(p_repeats_bed, d=True,t='last').to_dataframe().groupby(17).mean()[18].sort_values().tail(10)

In [ ]:
p_effector_bed.closest(p_repeats_bed, d=True,t='last').to_dataframe().groupby(17).mean()[18].sort_values().tail(10)

In [ ]:
tmp_REPET[(tmp_REPET.distance > 400)&(tmp_REPET.attributes == 'ClassII:?:?')].loc[:,gff_header].to_csv(tmp_fn, header=None, sep='\t', index=None)

In [ ]:
p_repeats_bed = BedTool(tmp_fn)

In [ ]:
print(p_effector_bed.closest(p_repeats_bed,d=True,t='last', io=True).to_dataframe().iloc[:,18:20].describe())
p_effector_bed.closest(p_repeats_bed, d=True, t='last', io=True).to_dataframe().boxplot(column=18)

In [ ]:
print(p_busco_bed.closest(p_repeats_bed, d=True,t='last', io=True).to_dataframe().iloc[:,18:20].describe())
p_busco_bed.closest(p_repeats_bed, d=True,t='last', io=True).to_dataframe().boxplot(column=18)

In [ ]:
print(p_noeffector_bed.closest(p_repeats_bed, d=True,t='last', io=True).to_dataframe().iloc[:,18:20].describe())
p_noeffector_bed.closest(p_repeats_bed, d=True,t='last', io=True).to_dataframe().boxplot(column=18)

In [ ]:
print(p_noeffector_bed.closest(p_repeats_bed, d=True,t='last').to_dataframe().iloc[:,18:20].describe())
p_noeffector_bed.closest(p_repeats_bed, d=True,t='last').to_dataframe().boxplot(column=18)

In [ ]:
p_noeffector_bed.closest(p_repeats_bed, d=True,t='last', io=True).to_dataframe().loc[:,18].plot(kind='density')
p_effector_bed.closest(p_repeats_bed, d=True,t='last', io=True).to_dataframe().loc[:,18].plot(kind='density', color='r')
p_busco_bed.closest(p_repeats_bed, d=True,t='last', io=True).to_dataframe().loc[:,18].plot(kind='density', color='g')
plt.semilogx()

In [ ]:
p_noeffector_bed.closest(p_repeats_bed, d=True,t='last').to_dataframe().loc[:,18].plot(kind='density')
p_effector_bed.closest(p_repeats_bed, d=True,t='last').to_dataframe().loc[:,18].plot(kind='density', color='r')
p_busco_bed.closest(p_repeats_bed, d=True,t='last').to_dataframe().loc[:,18].plot(kind='density', color='g')
plt.semilogx()

In [ ]:
tmp_REPET = pd.read_csv(p_repeat_superfamily_gff_fn, header=None, sep='\t', names=gff_header)
tmp_REPET['distance'] = tmp_REPET.end - tmp_REPET.start
tmp_fn = p_repeat_superfamily_gff_fn.replace('superfamily', 'tmpsuperfamily')
tmp_REPET[tmp_REPET.distance > 400].loc[:,gff_header].to_csv(tmp_fn, header=None, sep='\t', index=None)

p_repeats_bed = BedTool(tmp_fn)

In [ ]:
p_br_closest_df = p_busco_bed.closest(p_repeats_bed, d=True,t='last', io=True).to_dataframe(names = bed_repeat_closest_header)

p_br_closest_pt = p_br_closest_df.pivot_table(values='distance', index=['attributes_repeat'] ,aggfunc=[ np.count_nonzero, np.mean]).T.unstack().T

p_br_closest_pt['superfamily_%'] = p_br_closest_pt.count_nonzero / p_br_closest_pt.count_nonzero.sum() *100
old_columns =[x for x in p_br_closest_pt.columns] 
new_columns = [x+'_busco' for x in p_br_closest_pt.columns]
p_br_closest_pt.rename(columns=dict(zip(old_columns,new_columns)), inplace = True)

p_er_closest_df = p_effector_bed.closest(p_repeats_bed, d=True,t='last', io=True).to_dataframe(names = bed_repeat_closest_header)

p_er_closest_pt = p_er_closest_df.pivot_table(values='distance', index=['attributes_repeat'] ,aggfunc=[ np.count_nonzero, np.mean]).T.unstack().T

p_er_closest_pt['superfamily_%'] = p_er_closest_pt.count_nonzero / p_er_closest_pt.count_nonzero.sum() *100
old_columns =[x for x in p_er_closest_pt.columns] 
new_columns = [x+'_effector' for x in p_er_closest_pt.columns]
p_er_closest_pt.rename(columns=dict(zip(old_columns,new_columns)), inplace = True)

p_ner_closest_df = p_noeffector_bed.closest(p_repeats_bed, d=True,t='last', io=True).to_dataframe(names = bed_repeat_closest_header)

p_ner_closest_pt = p_ner_closest_df.pivot_table(values='distance', index=['attributes_repeat'] ,aggfunc=[ np.count_nonzero, np.mean]).T.unstack().T

p_ner_closest_pt['superfamily_%'] = p_ner_closest_pt['count_nonzero'] / p_ner_closest_pt['count_nonzero'].sum() *100
old_columns =[x for x in p_ner_closest_pt.columns] 
new_columns = [x+'_noeffector' for x in p_ner_closest_pt.columns]
p_ner_closest_pt.rename(columns=dict(zip(old_columns,new_columns)), inplace = True)

pd.options.display.float_format = '{:.2f}'.format

repeat_distance_df = pd.concat([p_ner_closest_pt.iloc[:,1:], p_br_closest_pt.iloc[:,1:], p_er_closest_pt.iloc[:,1:]], axis=1)

repeat_distance_df[repeat_distance_df['superfamily_%_effector'] >1].sort_values('superfamily_%_effector')

In [ ]:
#summary not allowing for overlaps of closest TEs not allowing for overlaps
p_br_closest_df = p_busco_bed.closest(p_repeats_bed, d=True,t='last').to_dataframe(names = bed_repeat_closest_header)

p_br_closest_pt = p_br_closest_df.pivot_table(values='distance', index=['attributes_repeat'] ,aggfunc=[ np.count_nonzero, np.mean]).T.unstack().T

p_br_closest_pt['superfamily_%'] = p_br_closest_pt.count_nonzero / p_br_closest_pt.count_nonzero.sum() *100
old_columns =[x for x in p_br_closest_pt.columns] 
new_columns = [x+'_busco' for x in p_br_closest_pt.columns]
p_br_closest_pt.rename(columns=dict(zip(old_columns,new_columns)), inplace = True)

p_er_closest_df = p_effector_bed.closest(p_repeats_bed, d=True,t='last').to_dataframe(names = bed_repeat_closest_header)

p_er_closest_pt = p_er_closest_df.pivot_table(values='distance', index=['attributes_repeat'] ,aggfunc=[ np.count_nonzero, np.mean]).T.unstack().T

p_er_closest_pt['superfamily_%'] = p_er_closest_pt.count_nonzero / p_er_closest_pt.count_nonzero.sum() *100
old_columns =[x for x in p_er_closest_pt.columns] 
new_columns = [x+'_effector' for x in p_er_closest_pt.columns]
p_er_closest_pt.rename(columns=dict(zip(old_columns,new_columns)), inplace = True)

p_ner_closest_df = p_noeffector_bed.closest(p_repeats_bed, d=True,t='last').to_dataframe(names = bed_repeat_closest_header)

p_ner_closest_pt = p_ner_closest_df.pivot_table(values='distance', index=['attributes_repeat'] ,aggfunc=[ np.count_nonzero, np.mean]).T.unstack().T

p_ner_closest_pt['superfamily_%'] = p_ner_closest_pt['count_nonzero'] / p_ner_closest_pt['count_nonzero'].sum() *100
old_columns =[x for x in p_ner_closest_pt.columns] 
new_columns = [x+'_noeffector' for x in p_ner_closest_pt.columns]
p_ner_closest_pt.rename(columns=dict(zip(old_columns,new_columns)), inplace = True)

pd.options.display.float_format = '{:.2f}'.format

repeat_distance_df = pd.concat([p_ner_closest_pt.iloc[:,1:], p_br_closest_pt.iloc[:,1:], p_er_closest_pt.iloc[:,1:]], axis=1)

repeat_distance_df[repeat_distance_df['superfamily_%_effector'] >1].sort_values('superfamily_%_effector')

Add some sub_sets for randommization to get equal sized groups

In [ ]:
#set the size of the subset here
sub_set = len(p_busco_bed)

In [ ]:
p_effector_bed = BedTool(p_effector_bed_fn)
p_allgene_bed = BedTool(p_gene_bed_fn)
p_busco_bed = BedTool(p_busco_bed_fn)
p_allall_rand_sub = p_allgene_bed.random_subset(sub_set)
p_effector_bed_rand_sub = p_effector_bed.random_subset(sub_set)
p_busco_bed_rand_sub = p_busco_bed.random_subset(sub_set)

In [ ]:
#get the distances with nearest gene effectors vs effectors
p_eself = p_effector_bed_rand_sub.closest(p_effector_bed_rand_sub, d=True,  N=True).to_dataframe().iloc[:,12]
p_eself = p_eself[p_eself > -1]
p_eself.name = 'Effectors'
p_eall = p_effector_bed_rand_sub.closest(p_allall_rand_sub, d=True,  N=True).to_dataframe().iloc[:,12]
p_eall= p_eall[p_eall > -1]
print(p_eself.describe())
p_eself.plot(kind='box')

In [ ]:
#get the distances with nearest gene all vs all subsampled
p_allall = p_allall_rand_sub.closest(p_allall_rand_sub, d=True, N=True).to_dataframe().iloc[:,12]
p_allall = p_allall[p_allall > -1]
p_allall.name = 'All_genes'
print(p_allall.describe())
p_allall.plot(kind='box')

In [ ]:
#now with buscos
p_bself = p_busco_bed_rand_sub.closest(p_busco_bed_rand_sub, d=True,  N=True).to_dataframe().iloc[:,12]
p_bself = p_bself[p_bself > -1]
p_bself.name = 'BUSCO'
print(p_bself.describe())
p_bself.plot(kind='box')

In [ ]:
#non_effectors
p_noeffector_bed= BedTool(p_noeffector_bed_fn)
p_noeffector_rand_sub = p_noeffector_bed.random_subset(sub_set)
p_neself = p_noeffector_rand_sub.closest(p_noeffector_rand_sub, d=True,  N=True).to_dataframe().iloc[:,12]
p_neself = p_neself[p_neself > -1]
p_neself.name = 'No_effectors'
print(p_neself.describe())
p_neself.plot(kind='box')

In [ ]:
import seaborn as sns

In [ ]:
sns.distplot(p_bself[p_bself < 10000], bins=20, rug = True, color='g',label='BUSCO')
sns.distplot(p_eself[p_eself < 10000], bins=20, rug = True, color='red',label = 'effector')
sns.distplot(p_neself[p_neself <10000], bins=20, rug=True, color='yellow', label ='not_effectors')
plt.legend()

In [ ]:
sns.distplot(p_bself[p_bself < 200000], bins=20, color='g' ,label='BUSCO')
sns.distplot(p_eself[p_eself < 200000], bins=20,  color='red',label = 'effector')
sns.distplot(p_neself[p_neself <200000], bins=20,  color='yellow', label ='not_effectors', axlabel='Distance to closest neighbour [bp]')
plt.legend()

In [ ]:
sns.distplot(p_bself, bins=20,color='g',  label='BUSCO')
sns.distplot(p_eself, bins=20,  color='red',label = 'effector')
sns.distplot(p_allall, bins=20,  color='b',label = 'all')
sns.distplot(p_neself, bins=20,  color='yellow', label ='not_effectors', axlabel='Distance to closest neighbour [bp]')
plt.legend()

In [ ]:
#now make a nearest neightbour dataframe
nn_df = pd.concat([p_allall, p_bself, p_eself], names=['All_genes', 'BUSCO', 'effectors'], axis=1)

In [ ]:
sns.violinplot(data=nn_df, palette="Set3")

In [ ]:
#NOW FILTER BY QUANTIELS

In [ ]:
low = 0
high = 0.8
quant_df = nn_df.quantile([low,high])

qfilt_nn_df = nn_df.apply(lambda x: x[(x > quant_df.loc[low, x.name]) & (x  < quant_df.loc[high, x.name])], axis=0)

sns.violinplot(data=qfilt_nn_df , palette="Set3")

In [ ]:
#now filter on IQR

In [ ]:
iqr_df_low = nn_df.apply(lambda x: x.quantile(0.25) - 1.5*(x.quantile(0.75) - x.quantile(0.25)) )
iqr_df_low.name ='low'
iqr_df_high = nn_df.apply(lambda x: x.quantile(0.75) + 1.5*(x.quantile(0.75) - x.quantile(0.25)) )
iqr_df_high.name = 'high'

iqr_df = pd.concat([iqr_df_low, iqr_df_high], axis=1).T

iqr_nn_df = nn_df.apply(lambda x: x[(x > iqr_df.loc['low', x.name]) & (x  < iqr_df.loc['high', x.name])], axis=0)
plt.title('Violine plot of nearest neighbour in the same category')
sns.violinplot(data=iqr_nn_df  , palette="Set3")

In [ ]:
p_bself[p_bself < 20000].plot(kind='density')
p_eself[p_eself < 20000].plot(kind='density', color='g')
p_neself[p_neself < 20000].plot(kind='density', color='r')
p_eall[p_eall < 20000].plot(kind='density', color='y')
#plt.semilogx()
print(len(p_bself[p_bself < 20000]), len(p_eself[p_eself < 20000]), len(p_neself[p_neself < 20000]), len(p_eall[p_eall < 20000]))
plt.xlim(0, 20000)

In [ ]:
p_eall[p_eall < 20000].plot(kind='hist', bins=20, normed=True)
plt.ylim(0, 0.00020)

In [ ]:
p_eself[p_eself < 20000].plot(kind='hist', bins=20, normed=True)
plt.ylim(0, 0.00020)

In [ ]:
p_bself[p_bself < 20000].plot(kind='hist', bins=20,normed=True)
plt.ylim(0, 0.00020)

In [ ]:
p_neself[p_neself < 20000].plot(kind='hist',bins=20,normed=True)
plt.ylim(0, 0.00020)

It seems like there seems to be no clear link between effector candidates and closest neighbour in terms of TEs in general. They all have the same distance in general.
Maybe Gypsy and ClassII:?:? should be looked at more carefully. Those are depleted and enriched in busco and effector genes. In general there seems to be a trend towards ClassII elements compared to ClassI in effector candidates.
This changes when for allowing for overlaps.

It seems that effectors are closer together than noneffector genes. Buscos also seem to cluster a bit. Let's see if we can visualize the location of genes on certain contigs vs repeats and such.

In [ ]:
#start with getting effectors per contig divided by length divded by # of overall genes

In [ ]:
effector_bdf = pd.read_csv(p_effector_bed_fn, header=None, sep='\t')

In [ ]:
p_effectors_per_contig  = effector_bdf.groupby(0).count()[1]
p_effectors_per_contig.name = 'effectors'

In [ ]:
p_all_genes_per_contig = pd.read_csv(p_gene_bed_fn, header=None, sep='\t').groupby(0).count()[1]
p_all_genes_per_contig.name = 'all_genes'
p_noeffectors_per_contig = pd.read_csv(p_noeffector_bed_fn, header=None, sep='\t').groupby(0).count()[1]
p_noeffectors_per_contig.name = 'no_effectors'
p_busco_per_contig = pd.read_csv(p_busco_bed_fn, header=None, sep='\t').groupby(0).count()[1]
p_busco_per_contig.name = 'buscos'

In [ ]:
p_contig_length = pd.read_csv(os.path.join(GFF_FOLDER, 'Pst_104E_v12_p_ctg.genome_file'), header = None,\
                          names=['contig' , 'length'], sep='\t').sort_values('contig')

In [ ]:
p_contig_length.index = p_contig_length.contig

In [ ]:
p_contig_length = p_contig_length.loc[:, 'length']

In [ ]:
gene_dis_per_contig = pd.concat([p_contig_length,p_all_genes_per_contig,p_noeffectors_per_contig,p_busco_per_contig,  p_effectors_per_contig, ], axis = 1)

In [ ]:
from scipy.stats import chisquare

In [ ]:
gene_dis_per_contig.fillna(value=0, inplace=True)

In [ ]:
gene_dis_per_contig.effectors.sum()

In [ ]:
#chisquare for genes per contig vs bases per contig
chisquare(gene_dis_per_contig.length.values/gene_dis_per_contig.length.sum()\
          , gene_dis_per_contig.all_genes.values/gene_dis_per_contig.all_genes.sum())

In [ ]:
chisquare(gene_dis_per_contig.buscos.values/gene_dis_per_contig.buscos.sum()\
          , gene_dis_per_contig.all_genes.values/gene_dis_per_contig.all_genes.sum())

In [ ]:
gene_dis_per_contig['%busco'] = gene_dis_per_contig.buscos.values/gene_dis_per_contig.buscos.sum() *100
gene_dis_per_contig['%effector'] = gene_dis_per_contig.effectors.values/gene_dis_per_contig.effectors.sum() *100
gene_dis_per_contig['%all_genes'] = gene_dis_per_contig.all_genes.values/gene_dis_per_contig.all_genes.sum() *100
gene_dis_per_contig['%no_effector'] = gene_dis_per_contig.no_effectors.values/gene_dis_per_contig.no_effectors.sum() *100

In [ ]:
chisquare(gene_dis_per_contig['%effector'], f_exp = gene_dis_per_contig['%all_genes'])

In [ ]:
gene_dis_per_contig.loc[:,['%effector', '%all_genes','%no_effector' ,'%busco']]

In [ ]:
gene_dis_per_contig[(gene_dis_per_contig['%effector'] > gene_dis_per_contig['%all_genes']) &\
                    (gene_dis_per_contig['%effector'] > gene_dis_per_contig['%busco'])].loc[:,['effectors', '%effector', '%all_genes']].index

In [ ]:
#effectors per contig > than expected
gene_dis_per_contig[(gene_dis_per_contig['%effector'] / gene_dis_per_contig['%all_genes']) > 1.2].loc[:,['effectors', '%effector', 'all_genes','%all_genes']].index

In [ ]:
#sum off all effectors on enriched? contigs
gene_dis_per_contig[(gene_dis_per_contig['%effector'] > gene_dis_per_contig['%all_genes']) &\
                    (gene_dis_per_contig['%effector'] > gene_dis_per_contig['%busco'])].sum()

In [ ]:
#sum off all effectors on enriched? contigs
gene_dis_per_contig[(gene_dis_per_contig['%effector'] > gene_dis_per_contig['%all_genes']) ].sum()

In [ ]:
gene_dis_per_contig[(gene_dis_per_contig['%busco'] / gene_dis_per_contig['%all_genes'] > 1.2) & (gene_dis_per_contig['%busco'] > 0.5 )\
                    ].loc[:,['buscos', '%busco', '%all_genes']]

In [ ]:
gene_dis_per_contig[(gene_dis_per_contig['%busco'] / gene_dis_per_contig['%all_genes'] > 1.2) \
                    ].sum()

In [ ]:
gene_dis_per_contig[(gene_dis_per_contig['%busco'] / gene_dis_per_contig['%all_genes'] > 1.1) & \
                   (gene_dis_per_contig['%effector'] / gene_dis_per_contig['%all_genes'] < 0.9) ].sum()

In [ ]:
gene_dis_per_contig[(gene_dis_per_contig['%busco'] / gene_dis_per_contig['%all_genes'] > 1.0000001) & \
                   (gene_dis_per_contig['%effector'] / gene_dis_per_contig['%all_genes'] < 0.9999) ].sum()

In [ ]:
#enriched for effectors relative to busco and all genes. Seems to be the case that their might be contigs
#with less BUSCOs and more effectors
gene_dis_per_contig[(gene_dis_per_contig['%busco'] / gene_dis_per_contig['%effector'] < 1) & \
                    (gene_dis_per_contig['%all_genes'] / gene_dis_per_contig['%effector'] < 1) ].sum()

Get closest features genes vs. TEs

In [ ]:
tmp_REPET = pd.read_csv(p_repeat_superfamily_gff_fn, header=None, sep='\t', names=gff_header)
tmp_REPET['distance'] = tmp_REPET.end - tmp_REPET.start

tmp_fn = p_repeat_superfamily_gff_fn.replace('superfamily.gff', 'tmpsuperfamily.bed')
tmp_REPET[tmp_REPET.distance > 400].loc[:,['seqid', 'start', 'end', 'attributes', 'score', 'strand']].to_csv(tmp_fn, header=None, sep='\t', index=None)

p_repeats_bed = BedTool(tmp_fn)

In [ ]:
tmp_df = p_effector_bed.closest( [p_repeats_bed.fn,p_allgene_bed.fn] , mdb='all', d=True, N=True).to_dataframe()

tmp_df.rename(columns={10: 'ID'}, inplace=True)

print('Per of effectors having genes as closest feature %f2' % (tmp_df[tmp_df.ID.str.contains('evm')][0].count()/len(p_effector_bed)*100))

print('Per of effectors having TE as closest %f2' % (tmp_df[~tmp_df.ID.str.contains('evm')][0].count()/len(p_effector_bed)*100))


In [ ]:
tmp_df = p_allgene_bed.closest( [p_repeats_bed.fn,p_allgene_bed.fn] , mdb='all', d=True, N=True).to_dataframe()

tmp_df.rename(columns={10: 'ID'}, inplace=True)

print('Per of genes having genes as closest feature %f2' % (tmp_df[tmp_df.ID.str.contains('evm')][0].count()/len(p_allgene_bed)*100))

print('Per of genes having TE as closest %f2' % (tmp_df[~tmp_df.ID.str.contains('evm')][0].count()/len(p_allgene_bed)*100))

Get 5 and 3 prime distances of genes

In [ ]:
#now get the 3 prime distante to genes using the D='a' and iu flag in bedtools
g_to_g_3 = p_allgene_bed.closest( p_allgene_bed.fn ,  N=True, iu=True, D='a' ).to_dataframe().iloc[:,[0, 3, 9,12]]


g_to_g_3.rename(columns={12:'3_distance', 3:'query', 9:'3_target', 0:'contig'}, inplace=True)

#now get the 5 prime distante to genes using the D='a' and id flag in bedtools
g_to_g_5 = p_allgene_bed.closest( p_allgene_bed.fn ,  N=True, id=True, D='a' ).to_dataframe().iloc[:,[0, 3, 9,12]]
g_to_g_5.rename(columns={12:'5_distance', 3:'query', 9:'5_target', 0:'contig'}, inplace=True)

g_to_g_merged = g_to_g_3.merge(g_to_g_5, on=['query', 'contig'])

#needs to be fixed to take boundaries into account. Filter out everythign that has a 5' distance of -1 meaning nothing
#and is at the begining of the contig .1 and at the 3' end drop everything that has a negative distance meaning no 3' neighbour

g_to_g_merged = g_to_g_merged[ (g_to_g_merged['5_target'] !='.') &(g_to_g_merged['3_target'] !='.') ]


g_to_g_merged['5_distance'] = abs(g_to_g_merged['5_distance'])
g_to_g_merged['5_distance_log10'] = np.log10(g_to_g_merged['5_distance'])
g_to_g_merged['3_distance_log10'] = np.log10(g_to_g_merged['3_distance'])

sns.jointplot(x='3_distance', y='5_distance', data=g_to_g_merged, kind="kde")



#do the same for effectors
#now for effectors 
#getting 5' and 3' distance
e_to_g_3 = p_effector_bed.closest( p_allgene_bed.fn ,  N=True, iu=True, D='a' ).to_dataframe().iloc[:,[0, 3, 9,12]]
e_to_g_3.rename(columns={12:'3_distance', 3:'query', 9:'3_target', 0:'contig'}, inplace=True)
e_to_g_5 = p_effector_bed.closest( p_allgene_bed.fn ,  N=True, id=True, D='a' ).to_dataframe().iloc[:,[0, 3, 9,12]]
e_to_g_5.rename(columns={12:'5_distance', 3:'query', 9:'5_target', 0:'contig'}, inplace=True)

#merging them
e_to_g_merged = e_to_g_3.merge(g_to_g_5, on=['query', 'contig'])

#needs to be fixed to take boundaries into account
#remove genes on the edges
e_to_g_merged = e_to_g_merged[((e_to_g_merged['5_target'] != '.') & ((e_to_g_merged['3_target'] != '.'))) ]
e_to_g_merged['5_distance'] = abs(e_to_g_merged['5_distance'])

sns.jointplot(x='3_distance', y='5_distance', data=e_to_g_merged, kind="kde")


#now for busco
b_to_g_3 = p_busco_bed.closest( p_allgene_bed.fn ,  N=True, iu=True, D='a' ).to_dataframe().iloc[:,[0, 3, 9,12]]
b_to_g_3.rename(columns={12:'3_distance', 3:'query', 9:'3_target', 0:'contig'}, inplace=True)
b_to_g_5 = p_busco_bed.closest( p_allgene_bed.fn ,  N=True, id=True, D='a' ).to_dataframe().iloc[:,[0, 3, 9,12]]
b_to_g_5.rename(columns={12:'5_distance', 3:'query', 9:'5_target', 0:'contig'}, inplace=True)
b_to_g_merged = b_to_g_3.merge(b_to_g_5, on=['query', 'contig'])

#needs to be fixed to take boundaries into account
#remove genes on the edges see above for details
b_to_g_merged = b_to_g_merged[((b_to_g_merged['5_target'] !='.') & (b_to_g_merged['3_target'] !='.')) ]
b_to_g_merged['5_distance'] = abs(b_to_g_merged['5_distance'])
sns.jointplot(x='3_distance', y='5_distance', data=b_to_g_merged, kind="kde")

#now start plotting stuff
#sns.jointplot(x='3_distance', y='5_distance', data=e_to_g_merged, kind="kde", color='r', xlim=10000, ylim=10000)
#sns.jointplot(x='3_distance', y='5_distance', data=g_to_g_merged, kind="kde", xlim=10000, ylim=10000)


#subset everything by fixed numbers maybe to IQR or such in future
sns.jointplot(x='3_distance', y='5_distance', data=e_to_g_merged[(e_to_g_merged['5_distance'] < 10000) &\
        (e_to_g_merged['3_distance'] < 10000)], kind="hex", color='r',xlim=[0,10000], ylim=[0,10000],\
             marginal_kws=dict(bins=30))


sns.jointplot(x='3_distance', y='5_distance', data=b_to_g_merged[(b_to_g_merged['5_distance'] < 10000) &\
        (b_to_g_merged['3_distance'] < 10000)], kind="hex",color='g', xlim=[0,10000], ylim=[0,10000],\
              marginal_kws=dict(bins=30))

sns.jointplot(x='3_distance', y='5_distance', data=g_to_g_merged[(g_to_g_merged['5_distance'] < 10000) &\
        (g_to_g_merged['3_distance'] < 10000)], kind="hex", xlim=[0,10000], ylim=[0,10000],\
              marginal_kws=dict(bins=30))

Now look at the distance to the closest gene of the same group

In [ ]:
#now get the 3 prime distante to genes using the D='a' and iu flag in bedtools
p_allall_rand_sub = p_allgene_bed.random_subset(sub_set)
p_effector_bed_rand_sub = p_effector_bed.random_subset(sub_set)
p_busco_bed_rand_sub = p_busco_bed.random_subset(sub_set)

all_all_rand_3 = p_allall_rand_sub.closest( p_allall_rand_sub ,  N=True, iu=True, D='a' ).to_dataframe().iloc[:,[0, 3, 9,12]]


all_all_rand_3.rename(columns={12:'3_distance', 3:'query', 9:'3_target', 0:'contig'}, inplace=True)

#now get the 5 prime distante to genes using the D='a' and id flag in bedtools
all_all_rand_5 = p_allall_rand_sub.closest( p_allall_rand_sub.fn ,  N=True, id=True, D='a' ).to_dataframe().iloc[:,[0, 3, 9,12]]
all_all_rand_5.rename(columns={12:'5_distance', 3:'query', 9:'5_target', 0:'contig'}, inplace=True)

all_all_rand_merged = all_all_rand_3.merge(all_all_rand_5, on=['query', 'contig'])

#needs to be fixed to take boundaries into account. Filter out everythign that has a 5' distance of -1 meaning nothing
#and is at the begining of the contig .1 <- the .1 was no appropriate here as the random subsampling can lead
#to alternative first gene models selected on the respective contig
#extended the initial df of closest to include also queyr and target. Now filtering on no target ='.' possible

all_all_rand_merged = all_all_rand_merged[((all_all_rand_merged['5_target'] != '.')&(all_all_rand_merged['3_target'] != '.')  ) ]



all_all_rand_merged['5_distance'] = abs(all_all_rand_merged['5_distance'])
all_all_rand_merged['5_distance_log10'] = np.log10(all_all_rand_merged['5_distance'])
all_all_rand_merged['3_distance_log10'] = np.log10(all_all_rand_merged['3_distance'])

sns.jointplot(x='3_distance', y='5_distance', data=all_all_rand_merged, kind="kde")



In [ ]:

#do the same for effectors
#now for effectors 
#getting 5' and 3' distance
e_to_e_sub_3 = p_effector_bed_rand_sub.closest( p_effector_bed_rand_sub ,  N=True, iu=True, D='a' ).to_dataframe()\
    .iloc[:,[0, 3, 9,12]]
e_to_e_sub_3.rename(columns={12:'3_distance', 3:'query', 9:'3_target', 0:'contig'}, inplace=True)
e_to_e_sub_5 = p_effector_bed_rand_sub.closest( p_effector_bed_rand_sub.fn ,  N=True, id=True, D='a' ).to_dataframe()\
    .iloc[:,[0, 3, 9,12]]
e_to_e_sub_5.rename(columns={12:'5_distance', 3:'query', 9:'5_target', 0:'contig'}, inplace=True)

#merging them
e_to_e_sub_merged = e_to_e_sub_3.merge(e_to_e_sub_5, on=['query', 'contig'])

#needs to be fixed to take boundaries into account
#remove genes on the edges
e_to_e_sub_merged = e_to_e_sub_merged [((e_to_e_sub_merged ['5_target'] != '.') & (e_to_e_sub_merged ['3_target'] != '.') )  ]
e_to_e_sub_merged ['5_distance'] = abs(e_to_e_sub_merged ['5_distance'])

sns.jointplot(x='3_distance', y='5_distance', data=e_to_e_sub_merged , kind="kde", color='red')



In [ ]:

#now for busco
p_busco_bed_rand_sub_3 = p_busco_bed_rand_sub.closest( p_busco_bed_rand_sub.fn ,  N=True, iu=True, D='a' ).to_dataframe().iloc[:,[0, 3, 9,12]]
p_busco_bed_rand_sub_3.rename(columns={12:'3_distance', 3:'query', 9:'3_target', 0:'contig'}, inplace=True)
p_busco_bed_rand_sub_5 = p_busco_bed_rand_sub.closest( p_busco_bed_rand_sub.fn ,  N=True, id=True, D='a' ).to_dataframe().iloc[:,[0, 3, 9,12]]
p_busco_bed_rand_sub_5.rename(columns={12:'5_distance', 3:'query', 9:'5_target', 0:'contig'}, inplace=True)
p_busco_bed_rand_sub_merged = p_busco_bed_rand_sub_3.merge(p_busco_bed_rand_sub_5, on=['query', 'contig'])

#needs to be fixed to take boundaries into account
#remove genes on the edges see above for details
p_busco_bed_rand_sub_merged = p_busco_bed_rand_sub_merged[((p_busco_bed_rand_sub_merged['5_target'] != '.') & (p_busco_bed_rand_sub_merged['5_target'] != '.') )]
p_busco_bed_rand_sub_merged['5_distance'] = abs(p_busco_bed_rand_sub_merged['5_distance'])
sns.jointplot(x='3_distance', y='5_distance', data=p_busco_bed_rand_sub_merged, kind="kde" ,color = 'green')

#now start plotting stuff
#sns.jointplot(x='3_distance', y='5_distance', data=e_to_g_merged, kind="kde", color='r', xlim=10000, ylim=10000)
#sns.jointplot(x='3_distance', y='5_distance', data=g_to_g_merged, kind="kde", xlim=10000, ylim=10000)




In [ ]:
#some more plotting for when subsetting the dataframe to exclude outliers
sns.jointplot(x='3_distance', y='5_distance', data=p_busco_bed_rand_sub_merged[(p_busco_bed_rand_sub_merged['5_distance'] < 20000)&\
                                                                              (p_busco_bed_rand_sub_merged['3_distance'] < 20000)],\
              kind="kde" ,color = 'green')

In [ ]:
#some more plotting for when subsetting the dataframe to exclude outliers
sns.jointplot(x='3_distance', y='5_distance', data=e_to_e_sub_merged[(e_to_e_sub_merged['5_distance'] < 20000)&\
                                                                              (e_to_e_sub_merged['3_distance'] < 20000)],\
              kind="kde" ,color = 'red')

In [ ]:
#looking more into the distance distribution between effectors use the index for this purpose and groupby or filteirng by 
#contig
#getting 5' and 3' distance


In [ ]:
#getting 5' and 3' distance
e_to_e_sub_distance = e_to_e_sub_3.merge(e_to_e_sub_5, on = ['query','contig'])

#convert negative -1 from bedtools closest to nan and make values absolute
tmp_index = e_to_e_sub_distance[e_to_e_sub_distance['5_target'] == '.'].index.tolist()
e_to_e_sub_distance.loc[tmp_index, '5_distance'] = np.nan
e_to_e_sub_distance['5_distance'] = abs(e_to_e_sub_distance['5_distance'])
#convert -1 from bedtools closest to nan in 3_distance
tmp_index = e_to_e_sub_distance[e_to_e_sub_distance['3_target'] == '.'].index.tolist()
e_to_e_sub_distance.loc[tmp_index, '3_distance'] = np.nan

#now subset the dataframe for a fixed distance 
max_distance = 15000
#subset the df and get the index
e_to_e_less_d_df = e_to_e_sub_distance[(e_to_e_sub_distance['3_distance'] <max_distance) | (e_to_e_sub_distance['5_distance'] <max_distance) ]
e_to_e_less_d_df_index = e_to_e_less_d_df.index.values


#introduce new column 'linked' and make this 1 were the genes are linked (e.g. less than max distance apart)
e_to_e_sub_distance['linked'] =0
e_to_e_sub_distance.loc[e_to_e_less_d_df_index, 'linked']  = 1
#get a new columns linkage_group that is 0 for now
e_to_e_sub_distance['linkage_group'] = 0


#now loop over the contigs and connect the consecutive linked effectors (e.g. linked == 1) with a cumsum 
for contig in e_to_e_sub_distance['contig'].unique():
    tmp_df = e_to_e_sub_distance[e_to_e_sub_distance.contig == contig]
    #check were the index difference is not one and add it up
    tmp_linkage_groups = (tmp_df[tmp_df.linked == 1].linked != tmp_df[tmp_df.linked == 1].linked.index.to_series().diff().eq(1)).cumsum() 
    e_to_e_sub_distance.loc[tmp_df[tmp_df.linked == 1].index, 'linkage_group'] =  tmp_linkage_groups
    #print(contig)
    

In [ ]:
#now fill in the nan values in the linkage_group 
e_to_e_sub_distance.fillna(0,inplace=True)

#the count of each linkage group per contig is the number of genes within this linkage group
lg_grouped = e_to_e_sub_distance.groupby(by=['contig', 'linkage_group']).linked.count()

#sum up the zeros these are the number of unlinked genes/effectors
number_unlinked_effectors = lg_grouped.unstack()[0].sum()

#the columns correspond to the name of the linkage group and the values to the number of genes contained in each
tmp_non_zero_df = lg_grouped.unstack().iloc[:,1:]

#count the number of each linkage group size for each abitrary linkage group and sum it up 
tmp_freq_count = tmp_non_zero_df.apply(pd.value_counts).sum(axis=1)

#this gives the frequency of each size of linkage group
tmp_freq_count = pd.Series([number_unlinked_effectors], index=[1.00]).append(tmp_freq_count)

In [ ]:
#now let's do the same for all genes
all_all_rand_distance = all_all_rand_3.merge(all_all_rand_5, on=['query', 'contig'])

#convert negative -1 from bedtools closest to nan and make values absolute
tmp_index = all_all_rand_distance[all_all_rand_distance['5_target'] == '.'].index.tolist()
all_all_rand_distance.loc[tmp_index, '5_distance'] = np.nan
all_all_rand_distance['5_distance'] = abs(all_all_rand_distance['5_distance'])
#convert -1 from bedtools closest to nan in 3_distance
tmp_index = all_all_rand_distance[all_all_rand_distance['3_target'] == '.'].index.tolist()
all_all_rand_distance.loc[tmp_index, '3_distance'] = np.nan

#now subset the dataframe for a fixed distance 
#max_distance = 10000
#subset the df and get the index
all_all_rand_less_d_df = all_all_rand_distance[(all_all_rand_distance['3_distance'] <max_distance) | \
                                               (all_all_rand_distance['5_distance'] <max_distance) ]
all_all_rand_less_d_df_index = all_all_rand_less_d_df.index.values


#introduce new column linked and make this 1 were the genes are linked (e.g. less than max distance apart)
all_all_rand_distance['linked'] =0
all_all_rand_distance.loc[all_all_rand_less_d_df_index, 'linked']  = 1
all_all_rand_distance['linkage_group'] = 0



for contig in all_all_rand_distance['contig'].unique():
    tmp_df = all_all_rand_distance[all_all_rand_distance.contig == contig]
    tmp_df['lg'] = 0
    #check were the index difference is not one and add it up
    tmp_linkage_groups = (tmp_df[tmp_df.linked == 1].linked != tmp_df[tmp_df.linked == 1].linked.index.to_series().diff().eq(1)).cumsum() 
    all_all_rand_distance.loc[tmp_df[tmp_df.linked == 1].index, 'linkage_group'] =  tmp_linkage_groups
    #print(contig)
all_all_rand_distance.fillna(0,inplace=True)

all_lg_grouped = all_all_rand_distance.groupby(by=['contig', 'linkage_group']).linked.count()

all_number_unlinked_effectors = all_lg_grouped.unstack()[0].sum()

all_tmp_non_zero_df = all_lg_grouped.unstack().iloc[:,1:]

all_tmp_freq_count = all_tmp_non_zero_df.apply(pd.value_counts).sum(axis=1)

all_tmp_freq_count = pd.Series([all_number_unlinked_effectors], index=[1.00]).append(all_tmp_freq_count)

In [ ]:
#getting 5' and 3' distance
busco_sub_distance = p_busco_bed_rand_sub_3.merge(p_busco_bed_rand_sub_5, on = ['query','contig'])

#convert negative -1 from bedtools closest to nan and make values absolute
tmp_index_busco = busco_sub_distance[busco_sub_distance['5_target'] == '.'].index.tolist()
busco_sub_distance.loc[tmp_index_busco, '5_distance'] = np.nan
busco_sub_distance['5_distance'] = abs(busco_sub_distance['5_distance'])
#convert -1 from bedtools closest to nan in 3_distance
tmp_index_busco = busco_sub_distance[busco_sub_distance['3_target'] == '.'].index.tolist()
busco_sub_distance.loc[tmp_index_busco, '3_distance'] = np.nan

#subset the df and get the index
busco_less_d_df = busco_sub_distance[(busco_sub_distance['3_distance'] <max_distance) | (busco_sub_distance['5_distance'] <max_distance) ]
busco_less_d_df_index = busco_less_d_df.index.values


#introduce new column 'linked' and make this 1 were the genes are linked (e.g. less than max distance apart)
busco_sub_distance['linked'] =0
busco_sub_distance.loc[busco_less_d_df_index, 'linked']  = 1
#get a new columns linkage_group that is 0 for now
busco_sub_distance['linkage_group'] = 0


#now loop over the contigs and connect the consecutive linked effectors (e.g. linked == 1) with a cumsum 
for contig in busco_sub_distance['contig'].unique():
    tmp_df = busco_sub_distance[busco_sub_distance.contig == contig]
    #check were the index difference is not one and add it up
    tmp_linkage_groups = (tmp_df[tmp_df.linked == 1].linked != tmp_df[tmp_df.linked == 1].linked.index.to_series().diff().eq(1)).cumsum() 
    busco_sub_distance.loc[tmp_df[tmp_df.linked == 1].index, 'linkage_group'] =  tmp_linkage_groups
    
#now fill in the nan values in the linkage_group 
busco_sub_distance.fillna(0,inplace=True)

#the count of each linkage group per contig is the number of genes within this linkage group
busco_lg_grouped = busco_sub_distance.groupby(by=['contig', 'linkage_group']).linked.count()

#sum up the zeros these are the number of unlinked genes/effectors
number_unlinked_buscos = busco_lg_grouped.unstack()[0].sum()

#the columns correspond to the name of the linkage group and the values to the number of genes contained in each
tmp_non_zero_df_busco = busco_lg_grouped.unstack().iloc[:,1:]

#count the number of each linkage group size for each abitrary linkage group and sum it up 
tmp_freq_count_busco = tmp_non_zero_df_busco.apply(pd.value_counts).sum(axis=1)

#this gives the frequency of each size of linkage group
tmp_freq_count_busco = pd.Series([number_unlinked_buscos], index=[1.00]).append(tmp_freq_count_busco)

In [ ]:
#generate a dataframe collecting all the frequencies
_df = pd.DataFrame([all_tmp_freq_count, tmp_freq_count, tmp_freq_count_busco]).T

In [ ]:
_df

In [ ]:
_df.rename(columns= {0:'all_genes', 1:'effectors', 2:'buscos'}, inplace=True)

In [ ]:
_df['effector_count'] = _df.index.values * _df.effectors
_df['gene_count'] = _df.index.values * _df.all_genes
_df['busco_count'] = _df.index.values * _df.buscos

In [ ]:
_df.loc[:,['effector_count', 'gene_count', 'busco_count']].plot()

In [ ]:
_df['group_size'] = _df.index.values

In [ ]:
_df['effector_cumsum']= ['effector_count'].cumsum()

In [ ]:
_df['gene_cumsum']= _df.sort_values(by='group_size', ascending=False)['gene_count'].cumsum()

In [ ]:
_df['busco_cumsum']= _df.sort_values(by='group_size', ascending=False)['busco_count'].cumsum()

In [ ]:
_df.sort_values(by='group_size', ascending=False).loc[:, ['effector_cumsum', 'gene_cumsum', 'busco_cumsum']].plot(colors=['r', 'b', 'g']\
                                                                                                                 )
plt.title('Number of genes in cluster with max nearest neigbhour distance = 24kpb')
plt.xlabel('cluster size')
plt.xlim(21,0)
plt.plot(( 0,17), ( 722,722), 'k-',label='Gene N50')
plt.legend()

In [ ]:
plt.scatter(x=_df.group_size.sort_values(ascending=False), y=_df.sort_values(by='group_size',\
                                                    ascending=False).loc[:, ['effector_cumsum']], color='r', label='effector_cumsum')
plt.scatter(x=_df.group_size.sort_values(ascending=False), y=_df.sort_values(by='group_size',\
                                                    ascending=False).loc[:, ['gene_cumsum']], color='b', label = 'gene_cumsum')
plt.scatter(x=_df.group_size.sort_values(ascending=False), y=_df.sort_values(by='group_size',\
                                                    ascending=False).loc[:, ['busco_cumsum']], color='g', label = 'busco_cumsum')
plt.xlim(21,0)
plt.legend(loc=4)
plt.title('Number of genes in cluster with max nearest neigbhour distance = %i' % max_distance)
plt.xlabel('cluster size')

In [ ]:
_df.sort_values(by='group_size', ascending=False).loc[:, ['effector_cumsum', 'gene_cumsum', 'busco_cumsum']].plot(colors=['r', 'b', 'g']\
                                                                                                                 )
plt.title('Number of genes in cluster with max nearest neigbhour distance = 24kpb')
plt.xlabel('cluster size')
plt.xlim(21,0)
plt.plot(( 0,17), ( 722,722), 'k-',label='Gene N50')
plt.legend()

In [ ]:
#look for the only cloned Avr homologe from Pgt
e_to_e_sub_distance[e_to_e_sub_distance['query'] == 'evm.TU.pcontig_013.278']

In [ ]:
e_to_e_sub_distance[(e_to_e_sub_distance.contig == 'pcontig_013') & (e_to_e_sub_distance.linkage_group == 3)]

In [ ]:
#look at the different distance dataframes to explore them further

In [ ]:
all_all_rand_merged[((all_all_rand_merged['5_distance'])< 10000)|((all_all_rand_merged['3_distance'])< 10000)].describe()

In [ ]:
p_busco_bed_rand_sub_merged[((p_busco_bed_rand_sub_merged['5_distance'])< 20000)|((p_busco_bed_rand_sub_merged['3_distance'])< 20000)].describe()

In [ ]:
e_to_e_sub_merged[(e_to_e_sub_merged['3_distance'] < 150000)|(e_to_e_sub_merged['5_distance'] < 150000)].describe()

In [ ]:
e_to_e_sub_3.describe()

In [ ]:
e_to_e_sub_5['abs_5_distance'] = abs(e_to_e_sub_5['5_distance'])

In [ ]:
e_to_e_sub_5.describe()

Quick look at allele analysis

In [ ]:
allele_QC_fn = os.path.join('/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/allele_analysis/alleles'\
                           , 'Pst_104E_v12_p_ctg.h_contig_overlap.Qcov80.PctID70.alleles')

allele_blast_df_fn = os.path.join('/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/allele_analysis' ,'Pst_104E_v12_p_ctg.Pst_104E_v12_h_ctg.0.001.blastp.outfmt6.allele_analysis')

alleles_df = pd.read_csv(allele_QC_fn, header=None, sep = '\t', names=['p_protein', 'h_protein'])
alleles_df['match'] = alleles_df.p_protein + alleles_df.h_protein

allele_blast_df = pd.read_csv(allele_blast_df_fn, sep='\t')
allele_blast_df['match'] = allele_blast_df.Query + allele_blast_df.Target

allele_blast_df = allele_blast_df[(allele_blast_df.match.isin(alleles_df.match))]

len(allele_blast_df)

allele_blast_df.head()

allele_blast_df[(allele_blast_df.Query.isin(p_effector_list))]['PctID'].mean()

allele_blast_df[~(allele_blast_df.Query.isin(p_effector_list))]['PctID'].mean()

print(len(allele_blast_df[(allele_blast_df.Query.isin(p_effector_list))])/len(p_effector_list))

print(len(allele_blast_df)/len(p_allgene_bed))